# Calibration example

In [ ]:
import pandas as pd
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.stats import bootstrap
from radiocalibrationtoolkit import *

In [ ]:
# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# some global plot settings
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = 16
plt.rcParams["legend.fontsize"] = 14

plt.rcParams["xtick.major.width"] = 2
plt.rcParams["ytick.major.width"] = 2

plt.rcParams["xtick.major.size"] = 5
plt.rcParams["ytick.major.size"] = 5

plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14

In [ ]:
# for this example you need to create a mock power dataframe and simulated sidereal power dataset

## Calibrate with "true"

In [ ]:
# in this case the recorded is the mock dataset
sky_map_model = 'LFmap'

power_rec_DF = pd.read_csv(
    "./mock_power_datasets/mock_power_dataset-Salla_EW_"+sky_map_model+"_N10000_temp-10_50C_0.0additionalnoise_rounding-True.csv",
    index_col=0,
)

power_rec_DF.columns = power_rec_DF.columns.astype(float)

dir_path = "./simulated_power_datasets/"
df_files = [
    os.path.join(dir_path, i) for i in os.listdir(dir_path) if (i.endswith(".csv") & i.startswith('Salla_EW'))
]
file_path_2_true = [i for i in df_files if sky_map_model in i][0]
power_sim_DF = pd.read_csv(file_path_2_true, index_col=0)
power_sim_DF.columns = power_sim_DF.columns.astype(float)

antenna_type = 'Salla_EW'
dir_path = "./simulated_power_datasets/"

concatenated_sim_df = concatenate_simulated_dfs(dir_path, antenna_type)

In [ ]:
# fit each band and make a plot overview
cmapT = plt.get_cmap("jet")
bounds = np.arange(30, 83, 2)
norm = mpl.colors.BoundaryNorm(bounds, cmapT.N)

fig, ax = plt.subplots(figsize=(8, 6))

frequencies_MHz = power_sim_DF.columns.values
slopes = []
intercepts = []
for i, freq in enumerate(power_sim_DF.columns):
    c = [freq] * power_rec_DF.index.size
    x_arr = power_sim_DF.loc[:, freq].values
    y_arr = power_rec_DF.loc[:, freq].values
    cs = ax.scatter(x_arr, y_arr, s=10, c=c, norm=norm, cmap="jet")

    intercept, slope = robust_regression(x_arr, y_arr)
    intercepts.append(intercept)
    slopes.append(slope)
    x_new = np.linspace(np.min(x_arr), np.max(x_arr), 100)
    ax.plot(
        x_new,
        x_new * slope + intercept,
        color=cmapT((freq - np.min(bounds)) * (bounds[1] - bounds[0]) / 100),
    )

intercepts = np.asarray(intercepts) ** (1 / 2)
slopes = np.asarray(slopes) ** (1 / 2)

cbar = fig.colorbar(cs, ticks=np.arange(30, 81, 4), ax=ax)
cbar.set_label("Frequency [MHz]")
ax.set_xlabel("simulated power [pW]")
ax.set_ylabel("measured (mock) power [pW]")
fig.subplots_adjust(left=0.15, bottom=0.2)

In [ ]:
fig_data = pickle.dumps(fig)
fig2 = pickle.loads(fig_data)

# modify the axis limits of the copied figure
ax2 = fig2.axes
ax2[0].set_ylim(2, 18)
fig2

In [ ]:
slopes_DF, intercepts_DF = get_fitted_voltage_calibration_params_and_noise_offsets(power_sim_DF, power_rec_DF)
get_and_plot_calibration_results(slopes_DF, intercepts_DF, title="", labels=None)

## Calibrate with all 'not true' simulated datasets

In [ ]:
df_list = []
df_names = []
except_this = 'Salla_EW_GSM16'
for f in df_files:
    if except_this not in f:
        df = pd.read_csv(f, index_col=0)
        df.columns = df.columns.astype(float)
        df_list.append(df)
        df_names.append(Path(f).stem)

concatenated_sim_df = pd.concat(df_list, keys=df_names)
# check keys
[key for key in concatenated_sim_df.index.levels[0]]

In [ ]:
(
    slopes_DF,
    intercepts_DF,
) = get_fitted_voltage_cal_params_and_noise_offsets_from_concat_sim_dfs(
    concatenated_sim_df, power_rec_DF
)
get_and_plot_calibration_results(slopes_DF, intercepts_DF, title=except_this)

## Iteratively apply the previous procedure to all simulated datasets

In [ ]:
# for this example you need to create a mock power dataframe and simulated sidereal power dataset

In [ ]:
galactic_models = [
    "GSM16",
    "LFSS",
    "GSM08",
    "Haslam",
    "LFmap",
    "SSM",
    "GMOSS",
    "ULSA",
]

all_freq_dep_cal_params_dict = {}
all_freq_dep_noise_dict = {}
central_stats_dict = {}
for gmodel in galactic_models:
    print("************************")
    print(gmodel)
    df_list = []
    df_names = []
    measured_label = "Salla_EW_" + gmodel
    for f in df_files:
        if measured_label not in f:
            df = pd.read_csv(f, index_col=0)
            df.columns = df.columns.astype(float)
            df_list.append(df)
            df_names.append(Path(f).stem)

    concatenated_sim_df = pd.concat(df_list, keys=df_names)
    power_rec_DF = pd.read_csv(
        "./mock_power_datasets/mock_power_dataset-"
        + measured_label
        + "_N10000_temp-10_50C_0.0additionalnoise_rounding-True.csv",
        index_col=0,
    )
    power_rec_DF.columns = power_rec_DF.columns.astype(float)
    (
        slopes_DF,
        intercepts_DF,
    ) = get_fitted_voltage_cal_params_and_noise_offsets_from_concat_sim_dfs(
        concatenated_sim_df, power_rec_DF
    )
    all_freq_dep_cal_params_dict[gmodel], all_freq_dep_noise_dict[gmodel] = (
        slopes_DF,
        intercepts_DF,
    )
    fig, ax = plt.subplots()
    central_stats_dict[gmodel] = get_and_plot_calibration_results(
        slopes_DF, intercepts_DF, title=except_this
    )
    print("************************")

In [ ]:
all_slopes_df = pd.concat(all_freq_dep_cal_params_dict)
all_intercepts_df = pd.concat(all_freq_dep_noise_dict)
central_stats_DF = pd.DataFrame(central_stats_dict, index=["mu", "err_low", "err_high"]).T

In [ ]:
get_and_plot_calibration_results(all_slopes_df, all_intercepts_df, title="", labels=None)

In [ ]:
fig, ax = plt.subplots()

ax.set_title('possible biases')
ax.hist(central_stats_DF.mu.values)
ax.set_xlabel('voltage calibration parameter')
ax.set_ylabel('Entries')

print('Min and Max observed bias:')
print(np.min(central_stats_DF.mu.values), np.max(central_stats_DF.mu.values))

In [ ]:
data = (central_stats_DF.mu.values,)  # samples must be in a sequence

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
fig, ax = plt.subplots(1,2, figsize=(12,4))
res = bootstrap(data, np.mean, confidence_level=0.9)
bvalues = res.bootstrap_distribution
ax[0].hist(bvalues, bins=25)
ax[0].set_title('boostraped')
ax[0].set_xlabel(r'$<bias>$')
ax[0].text(0.05, 0.95, '$\mu$={:.2f}'.format(np.mean(bvalues)), transform=ax[0].transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
ax[0].set_ylabel('entries')

ax[1].set_title('boostraped')
res = bootstrap(data, np.std, confidence_level=0.9)
bvalues = res.bootstrap_distribution
ax[1].hist(bvalues, bins=25)
ax[1].set_xlabel('$\sigma$ bias')
ax[1].text(0.05, 0.95, '$\mu$={:.2f}'.format(np.mean(bvalues)), transform=ax[1].transAxes, fontsize=14,
        verticalalignment='top', bbox=props)
ax[1].set_ylabel('entries')
fig.subplots_adjust(wspace=0.3)